In [1]:
import numpy as np
import tensorflowjs as tfjs

from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam


In [2]:
def generate_white_noise_image(height, width):
    # # Create an image with random values [0, 255) and scale to [0, 1)
    # arr = np.random.randint(0, 255, (height, width), dtype=np.uint8) / 255.0
    # # Set 400 random pixels to 0 (black)
    # for i in range(900):
    #     arr[np.random.randint(0, height), np.random.randint(0, width)] = 0
    # print(arr.shape)
    # return arr  # Return the modified array
    return np.zeros((height, width))

In [3]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
x_train = np.array([generate_white_noise_image(28, 28) if label == 0 else img for img, label in zip(x_train, y_train)])
x_test = np.array([generate_white_noise_image(28, 28) if label == 0 else img for img, label in zip(x_test, y_test)])
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [5]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [6]:
datagen.fit(x_train)

In [7]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [8]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last',
                 input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid' ))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu', data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [9]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 28, 28, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                        

In [10]:
batch_size = 64
epochs = 20

In [11]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size), epochs = epochs, 
                              validation_data = (x_test, y_test), verbose=1, 
                              steps_per_epoch=x_train.shape[0] // batch_size)

Epoch 1/8


/var/folders/vy/k9596k3d1p57g74qlj76v3h00000gn/T/ipykernel_72252/2662907208.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size), epochs = epochs,
2024-04-17 13:10:31.985693: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


937/937 [==============================] - 24s 24ms/step - loss: 0.2980 - accuracy: 0.9124 - val_loss: 0.0504 - val_accuracy: 0.9839
Epoch 2/8
937/937 [==============================] - 22s 23ms/step - loss: 0.1259 - accuracy: 0.9616 - val_loss: 0.0332 - val_accuracy: 0.9907
Epoch 3/8
937/937 [==============================] - 22s 23ms/step - loss: 0.1058 - accuracy: 0.9681 - val_loss: 0.0472 - val_accuracy: 0.9859
Epoch 4/8
937/937 [==============================] - 22s 23ms/step - loss: 0.0941 - accuracy: 0.9715 - val_loss: 0.0315 - val_accuracy: 0.9904
Epoch 5/8
937/937 [==============================] - 22s 23ms/step - loss: 0.0914 - accuracy: 0.9725 - val_loss: 0.0333 - val_accuracy: 0.9903
Epoch 6/8
937/937 [==============================] - 23s 24ms/step - loss: 0.0832 - accuracy: 0.9754 - val_loss: 0.0302 - val_accuracy: 0.9919
Epoch 7/8
937/937 [==============================] - 23s 25ms/step - loss: 0.0784 - accuracy: 0.9757 - val_loss: 0.0238 - val_accuracy: 0.9926
Epoch 8/8

In [12]:
tfjs.converters.save_keras_model(model, "./models/tfjs_model")

/Users/andreas/Documents/code/SudokuSolver/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
